In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

def preprocess_data(df):
    label_encoder = LabelEncoder()
    df['stroke'] = label_encoder.fit_transform(df['stroke'])
    for col in df.columns:
        if df[col].dtype == 'object' and col != 'stroke':
            df[col] = label_encoder.fit_transform(df[col])
    return df, label_encoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/DEEP LEARNING LAB/Datasets/health.csv")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [13]:
x = data.drop('stroke', axis =1)
y = data['stroke']

x_train,y_train,x_test, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [20]:
x_train

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
802,4970,Male,79.00,0,0,Yes,Self-employed,Rural,112.64,28.5,formerly smoked
3927,56137,Female,62.00,0,0,Yes,Private,Urban,88.32,36.3,Unknown
2337,54590,Female,21.00,0,0,No,Private,Rural,59.52,33.7,never smoked
3910,36548,Male,31.00,0,0,Yes,Govt_job,Urban,65.70,30.4,formerly smoked
1886,61171,Female,31.00,0,0,No,Private,Rural,59.63,19.9,never smoked
...,...,...,...,...,...,...,...,...,...,...,...
4426,13846,Male,43.00,0,0,Yes,Govt_job,Rural,88.00,30.6,never smoked
466,1307,Female,61.00,1,0,Yes,Private,Rural,170.05,60.2,smokes
3092,31481,Female,1.16,0,0,No,children,Urban,97.28,17.8,Unknown
3772,61827,Male,80.00,0,0,Yes,Self-employed,Rural,196.08,31.0,formerly smoked


In [24]:
X_test[0]

array([4.0041e+04, 1.0000e+00, 3.1000e+01, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 3.0000e+00, 0.0000e+00, 6.4850e+01, 0.0000e+00])

In [16]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.weights = np.zeros(X.shape[1])
        self.bias = 0
        for _ in range(self.epochs):
            model = np.dot(X, self.weights) + self.bias
            predictions = self.sigmoid(model)
            dw = (1 / X.shape[0]) * np.dot(X.T, (predictions - y))
            db = (1 / X.shape[0]) * np.sum(predictions - y)
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        model = np.dot(X, self.weights) + self.bias
        predictions = self.sigmoid(model)
        return np.round(predictions)

    def predict_single_instance(self, X_single):
      model = np.dot(X_single, self.weights) + self.bias
      prob_class_1 = self.sigmoid(model)
      prob_class_0 = 1 - prob_class_1
      return prob_class_0, prob_class_1

df = data
df, label_encoder = preprocess_data(df)

X = df.drop(columns=['stroke', 'bmi'], axis=1).values
y = df['stroke'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(learning_rate=0.01, epochs=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


<ipython-input-16-bbbff3122e44>:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Accuracy: 93.93%


In [26]:
X_single = X_test[0].reshape(1, -1)
prob_class_0, prob_class_1 = model.predict_single_instance(X_single)

print(f"Probability for class 0: {prob_class_0[0]:.4f}")
print(f"Probability for class 1: {prob_class_1[0]:.4f}")

Probability for class 0: 1.0000
Probability for class 1: 0.0000


<ipython-input-16-bbbff3122e44>:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [27]:
X_single

array([[4.0041e+04, 1.0000e+00, 3.1000e+01, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 3.0000e+00, 0.0000e+00, 6.4850e+01, 0.0000e+00]])